## 目标

本篇文章的目标是：探索“残差网络（ResNet）”开源代码，搞清楚它是如何工作的。

## 准备工作

本文不是入门文章，建议读者在阅读之前，先做好以下准备：
1. 学习吴恩达在coursera的“深度学习课程”中关于[残差网络的内容](https://www.coursera.org/learn/convolutional-neural-networks/home/welcome)
2. 读该模型的原版论文：[Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)，如果阅读有难度，可以参考网络上的翻译稿，这里有一篇笔者的[翻译稿](https://zhuanlan.zhihu.com/p/31428906)供参考。
3. 注册github，用于查看和下载残差网络的开源源码。[注册地址](https://github.com/)。
4. 复制源代码到本地。[源码地址在此](https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py)。

## 其他说明

- 笔者使用的操作系统是macOS Sierra-version 10.12.6
- Python 版本是Python 2.7.13

## 先测试模型效果

测试代码的话，先看看源码的说明文档。[地址在这里](https://github.com/fchollet/deep-learning-models)。
根据说明文档的描述，该repo包含以下模型：
- VGG16
- VGG19
- ResNet50
- Inception v3
- CRNN for music tagging

我们只关心ResNet模型，也就是这里的ResNet50。

继续看说明文档，有关于图片分类的示例代码，观察该代码，使用的是renet。这就是我们要测试的代码：

In [ ]:
from resnet50 import ResNet50
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions

model = ResNet50(weights='imagenet')

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds))
# print: [[u'n02504458', u'African_elephant']]

上述示例的第一句就是读取resnet50中的ResNet50，所以我们创建resnet50.py文件，并复制ResNet50的代码。

观察resnet50.py的执行代码与上述示例代码一致，也就是说我们可以直接运行该文档。
在存放resnet50.py的本地文档打开Terminal，然后运行resnet50.py：

In [ ]:
> python resnet50.py

**可惜报错了!!!**

In [ ]:
raceback (most recent call last):
  File "resnet50.py", line 289, in <module>
    model = ResNet50(include_top=True, weights='imagenet')
  File "resnet50.py", line 193, in ResNet50
    include_top=include_top)
TypeError: _obtain_input_shape() got an unexpected keyword argument 'include_top'

没关系，我们看一下什么错误？
首先找到出错的位置："resnet50.py"的193行。

In [ ]:
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=197,
                                      data_format=K.image_data_format(),
                                      include_top=include_top)

这一行就是错误信息中提到的`_obtain_input_shape()`函数。错误信息的意思是该函数有一个错误的参数：'include_top'？

OK，我们来查一下`_obtain_input_shape()`应该有哪些参数？以下是该函数的定义。

In [ ]:
def _obtain_input_shape(input_shape,  
                        default_size,  
                        min_size,  
                        data_format,  
                        require_flatten,  
                        weights=None):  

这里没有include_top参数，多了一个require_flatten参数。我们试一下将include_top改为require_flatten，再运行一下。

一个好消息和一个坏消息，`_obtain_input_shape()`函数报错消失了，但是又有一个新的报错：

In [ ]:
Traceback (most recent call last):
  File "resnet50.py", line 292, in <module>
    img = image.load_img(img_path, target_size=(224, 224))
  File "/Users/freefrog/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 322, in load_img
    img = pil_image.open(path)
  File "/Users/freefrog/anaconda2/lib/python2.7/site-packages/PIL/Image.py", line 2410, in open
    fp = builtins.open(filename, "rb")
IOError: [Errno 2] No such file or directory: 'elephant.jpg'

仔细看报错信息的最后一行，我们没有添加命名为'elephant.jpg'的图片，我们可以从网上下载一张图片，并取名为'elephant.jpg'，放在resnet50.py同一个文件夹下，然后再次运行代码：

如果你看到类似下面的返回值，那么恭喜你，脚本运行成功了！！！

In [ ]:
Input image shape: (1, 224, 224, 3)
Predicted: [[(u'n02504458', u'African_elephant', 0.53912073), (u'n01871265', u'tusker', 0.26061574), (u'n02504013', u'Indian_elephant', 0.13235191), (u'n02437312', u'Arabian_camel', 0.021120256), (u'n02109047', u'Great_Dane', 0.0058048805)]]

我们先大概看看返回的结果什么意思？
- `(u'n02504458', u'African_elephant', 0.53912073)`的意思应该是预测非洲象的概率是0.53912073。
- `(u'n01871265', u'tusker', 0.26061574)`的意思是‘有长牙的动物（tusker）’的概率是0.26061574。

再后面的返回值是印度象、阿拉伯骆驼、大丹犬的概率分别是多少。
从结果来看预测是非洲象的概率最大，预测成功！！！下面是我们使用的elephant.jpg图片。

![elephant.jpg](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1511889820050&di=eab2b182a52ebe42a707110070c2cda3&imgtype=0&src=http%3A%2F%2Fimgsrc.baidu.com%2Fbaike%2Fabpic%2Fitem%2Ffaacb5640d1faaa4f7365481.jpg)

## 看看代码结构

resnet50.py定义了三个函数：`identity_block`, `conv_block`和`ResNet50`。从名字上看，应该是恒等结构快、卷积结构块和resnet模型。两种结构块是构成模型的基本单元，这一点通过示例代码也可以发现（示例代码只调用了`ResNet50`函数）。

**为什么要定义两种结构块？**

在论文的3.3节Residual Network段落，作者提到，如果输入和输出层的维度一致，那么可以使用**恒等快捷通道**（对应`identity_block`结构块），但是，如果维度不一致的话，需要采取措施如**投影快捷连接**（对应代码中的`conv_block`结构块）。

既然`identity_block`, `conv_block`用于构建`ResNet50`，我们从函数的参数和返回值入手，弄清楚以下问题：
1. 构建resnet需要哪些参数，有什么作用？
2. 基本结构块需要哪些参数，有什么作用？

**首先来看一下`ResNet50`函数**

该函数定义如下：

In [ ]:
def ResNet50(include_top=True, weights='imagenet',
             input_tensor=None, input_shape=None,
             pooling=None,
             classes=1000):

该函数一共有6个参数，我们分别来看一下是什么意思（以下内容的英文原文在源代码的函数里）：

- include_top: 逻辑值，在网络架构的顶端（也就是最后），是否包含全连接层。
- weights: 二选一：`None`（代表随机初始化权重）或者"imagenet"（代表读入在ImageNet上预训练的权重）。
- input_tensor: 可选参数，Keras tensor（即`layers.Input()`的输出），作为模型的输入图片。
- input_shape: 可选参数，元组类型的维度，只有当`include_top`参数是False时，需要指定该参数，否则，输入图片的维度必须是`(224, 224, 3)`（`channels_last`格式）或`(3, 224, 244)`（`channels_first`格式），channels维度必须是3，宽度和高度大小必须大于197.
- pooling: 可选参数，特征图提取的池化模式，仅当`include_top`为`False`时指定该参数的值。可选项包括`None`（无池化），`avg`（平均池化），`max`（最大池化）。
- classes: 可选参数，指定图片分类的类别数量，该参数只有当`include_top`是'True'且没有指定`weights`参数时指定。

我们发现，其他很多参数都与‘include_top’参数是否已指定有关，那么‘include_top’指定的全连接层有什么作用呢？

简单来说，全连接层将学到的“分布式特征表示”映射到样本标记空间的作用。所以，如果不使用全连接层的话，我们可以指定池化的方式得到输出。如果使用全连接层的话，我们可以指定输出的类别的数量。
[读者可以参考](https://www.zhihu.com/question/41037974)

该函数的输出是一个Keras模型的实例。

搞清楚了`ResNet50`函数的输入输出，我们再回过头来看一下我们的实例代码中调用该函数的语句，有以下两句：

In [ ]:
model = ResNet50(weights='imagenet')
preds = model.predict(x)

第一句指定‘weights='imagenet'’，意思是读入预训练的权重，返回的model是Keras模型的实例，既然是Keras模型的实例就可以使用`.predict`函数。x是处理后的图片，也就是对图片x进行预测，返回预测值。

**再来看一下`identity_block`**函数

`identity_block`函数的定义如下：

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):

该函数一共有5个参数：

- input_tensor: 输入tensor
- kernel_size: 默认值3, 主路中间卷积层的卷积核大小
- filters: 实数列表，主路3个卷积层的过滤器
- stage: 实数，当前阶段标签，用于生成层名称。
- block: 'a'、'b'等，当前结构块标签，用于生成层名称。

函数的输出是tensor。

分析输入参数和返回值，我们对该函数的理解是，一个恒等结构块由三个卷积层和一个**恒等快捷通道**组成，正如论文中所说，**恒等快捷通道**不需要任何参数！我们需要指定的参数是卷积核大小、三个过滤器以及每一层的名称。

输入一个tensor给该结构块，经过三个卷积层和一个恒等快捷通道后，得到输出tensor。

**最后来看一下`conv_block`**函数

该函数的定义如下：

In [ ]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):

该函数包含6个参数：
- input_tensor: 输入tensor
- kernel_size: 默认值3, 主路中间卷积层的卷积核大小
- filters: 实数列表，主路3个卷积层的过滤器
- stage: 实数，当前阶段标签，用于生成层名称。
- block: 'a'、'b'等，当前结构块标签，用于生成层名称。
- strides=(2, 2): 对于阶段3，主路的第一个卷积层和快捷通道的strides都是（2，2）

函数的输出是tensor。

`conv_block`函数的输入参数比`identity_block`多了一个‘strides=(2, 2)’，强制将其设置为‘strides=(2, 2)’，为什么要这样做？我们在下一节进一步探索这三个函数是如何工作的。

分析输入参数和返回值，我们对该函数的理解是，一个卷积结构块由三个卷积层和一个卷积快捷通道组成，卷积快捷通道的作用是匹配输入和输出维度，具体是怎么做到的呢？我们接下来继续探索。

## 进一步探索

**主函数**

有了对三个函数的基本了解，下面进行进一步探索。这次我们从主函数开始，主函数是下面这个样子：

In [ ]:
if __name__ == '__main__':
    model = ResNet50(include_top=True, weights='imagenet')

    img_path = 'elephant.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    print('Input image shape:', x.shape)

    preds = model.predict(x)
    print('Predicted:', decode_predictions(preds))

源代码通过空行将这段代码分割为三部分，我们分别来分析一下。

*（注：`if __name__ == '__main__':`的作用这里不再赘述，如果不明白请百度，或者简单理解为：加了这一行该脚本既可以单独运行也可以被其他脚本调用，[参考这里](http://www.dengfeilong.com/post/60.html)。）*

第一部分只有一行代码，也很好理解：运行`ResNet50`函数。我们已经分析过该函数，得到的返回值是一个Keras模型的实例。这里代入的参数是`include_top=True, weights='imagenet'`，代表该模型有全连接层，且读入在ImageNet上预训练的权重。

In [ ]:
model = ResNet50(include_top=True, weights='imagenet')

第二部分共6行代码。

In [ ]:
    img_path = 'elephant.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    print('Input image shape:', x.shape)

我们依次来解读一下。

前两行很简单，设置图片地址为`img_path = 'elephant.jpg'`，也就是说，我们需要一张名称为‘elephant.jpg’的图片在本地文件夹。第二行就是读入图片并命名为变量img，参数也很好理解，分别是我们设置好的图片路径和图片大小，这里为什么要限制图片大小？我们在分析`ResNet50`函数的输入参数是提到过，如果该函数的include_top参数为True，我们的图片大小必须是（224，224）。

In [ ]:
    img_path = 'elephant.jpg'
    img = image.load_img(img_path, target_size=(224, 224))

这里，还需要注意的是，image.load_img()函数从哪里来呢？答案是`keras.preprocessing`。我们查看一下脚本前面的引入部分，有一行代码是：`from keras.preprocessing import image`。如果我们不使用该函数行不行呢？我们来做个实验：

首先引入`matplotlib`库和`scipy`库用于图片的读取和缩放。

In [ ]:
import matplotlib.image as mpimg
from scipy import misc

然后将`img = image.load_img(img_path, target_size=(224, 224))`替换为下面的代码：

In [ ]:
img = mpimg.imread(img_path) 
img = misc.imresize(img, [224,224])

我们再次运行脚本，同样能够得到预测结果，但是，结果的数值与原来不同了，为什么？

因为在对图片进行强制转换大小时，`misc.imresize()`和`image.load_img()`采用了不同的**差值**方法！

关于差值方法我们这里不深入讨论，感兴趣的读者可以显示这两种方法产生的图片感受一下。

可以使用如下测试代码：

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import misc
from keras.preprocessing import image

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
#img = mpimg.imread(img_path) 
#img = misc.imresize(img, [224,224])
x = image.img_to_array(img)

plt.imshow(img)
plt.show()

所以当我们使用keras库时，建议采用该库的`keras.preprocessing.image`.

继续看下一行。

In [ ]:
x = image.img_to_array(img)

该函数字面意思很容易理解，将图片转换格式为array，为了方便后续处理

In [ ]:
x = np.expand_dims(x, axis=0)

然后扩展array的维度，也就是从(224,224,3)扩展为(1,224,224,3)，为什么要这样么做呢，很简单，扩展的一个维度用于代表样本，假设我们有100张图片，那么就可以用这种4维的tensor一次性输入模型，而不是一张一张输入。

再来看最后一句：

In [ ]:
x = preprocess_input(x)

对x进行预处理，都进行哪些预处理呢？主要是对图片进行归一化处理。[preprocess_input的官方代码在这](https://github.com/fchollet/keras/blob/master/keras/applications/imagenet_utils.py)

前面的准备工作做完，下面进入主函数代码的第三部分。

In [ ]:
    preds = model.predict(x)
    print('Predicted:', decode_predictions(preds))

由于我们的代码不需要训练，使用的是预训练的权重，因此只需要直接将输入图片作为参数预测即可得到结果。

接下来进一步探索ResNet50函数。

**ResNet50**

为了简单起见，也为了跟代码思路同步，我们一起按照注释的划分进行代码分析。

注释一共有5处，将代码划分为5部分，我们依次来看一下。

*第一部分：错误警告*

代码的第一部分注释解释了该函数的作用以及参数定义，该内容我们已经分析过了，直接看最后一句注释和接下来的代码。

In [ ]:
    """
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')

这里定义了两种情况，发现这两种情况，函数将报错，这两种情况分别是：

- weights参数既不是‘imagenet’也不是‘None’
- 当weights是‘imagenet’，include_top是true，classes不等于1000

为什么是这两种情况的话，请看我们前面对该函数参数的分析。

另外这里需要记住的是`raise ValueError()`的用法，结合条件判断语句常用来对参数的有效性进行判断。

*第二部分：搭建算法框架*

此部分代码如下：